In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, desc, col
from pyspark.sql.functions import max
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import when
from pyspark.sql.functions import col, upper, when

In [3]:
#Build a spark session
spark = SparkSession.builder.appName("Severity_prediction").getOrCreate()

In [4]:
df = spark.read.csv("C:/Users/vmari/OneDrive/Desktop/Lambton/2nd_term/AI/Project_Accidents_severity/US_Accidents_March23.csv", header=True, inferSchema=False)



In [5]:
df.show()

+----+-------+--------+-------------------+-------------------+------------------+------------------+-------+-------+------------+--------------------+--------------------+------------+----------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
|  ID| Source|Severity|         Start_Time|           End_Time|         Start_Lat|         Start_Lng|End_Lat|End_Lng|Distance(mi)|         Description|              Street|        City|    County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|A

In [6]:
Description=df.select("Description")
Description.show(200,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------+
|Description                                                                                                                            |
+---------------------------------------------------------------------------------------------------------------------------------------+
|Right lane blocked due to accident on I-70 Eastbound at Exit 41 OH-235 State Route 4.                                                  |
|Accident on Brice Rd at Tussing Rd. Expect delays.                                                                                     |
|Accident on OH-32 State Route 32 Westbound at Dela Palma Rd. Expect delays.                                                            |
|Accident on I-75 Southbound at Exits 52 52B US-35. Expect delays.                                                                      |
|Accident on McEwen Rd at OH-725 M

Precipitation(in)= precipitation acumulation amount

In [7]:
Precipitation_exploaration=df.select("Precipitation(in)","Humidity(%)","Severity")
Precipitation_exploaration.show()

+-----------------+-----------+--------+
|Precipitation(in)|Humidity(%)|Severity|
+-----------------+-----------+--------+
|             0.02|       91.0|       3|
|              0.0|      100.0|       2|
|             NULL|      100.0|       2|
|             NULL|       96.0|       3|
|             NULL|       89.0|       2|
|             0.03|       97.0|       3|
|             NULL|      100.0|       2|
|             NULL|      100.0|       3|
|             NULL|       99.0|       2|
|             0.02|      100.0|       3|
|             NULL|       93.0|       3|
|             0.02|      100.0|       3|
|             NULL|      100.0|       2|
|             NULL|       89.0|       2|
|             0.02|      100.0|       2|
|             NULL|      100.0|       2|
|             NULL|       99.0|       2|
|             NULL|       89.0|       2|
|             NULL|       93.0|       2|
|             NULL|       89.0|       2|
+-----------------+-----------+--------+
only showing top

In [8]:
max_value_p = Precipitation_exploaration.select(max("Precipitation(in)")).collect()[0][0]
print("Max value of precipitation(in) =",max_value_p)



Max value of precipitation(in) = 9.99


In [9]:
value_counts_p = Precipitation_exploaration.groupBy("Precipitation(in)").count().orderBy("Precipitation(in)")

value_counts_p.show(200)

+-----------------+-------+
|Precipitation(in)|  count|
+-----------------+-------+
|             NULL|2203586|
|              0.0|4991718|
|             0.01| 151010|
|             0.02|  74008|
|             0.03|  50055|
|             0.04|  37300|
|             0.05|  29167|
|             0.06|  23958|
|             0.07|  19353|
|             0.08|  16138|
|             0.09|  14307|
|              0.1|  11916|
|             0.11|  10166|
|             0.12|   8805|
|             0.13|   7847|
|             0.14|   6775|
|             0.15|   5841|
|             0.16|   5169|
|             0.17|   4746|
|             0.18|   4222|
|             0.19|   3702|
|              0.2|   3348|
|             0.21|   2980|
|             0.22|   2654|
|             0.23|   2581|
|             0.24|   2510|
|             0.25|   1985|
|             0.26|   1949|
|             0.27|   1731|
|             0.28|   1693|
|             0.29|   1305|
|              0.3|   1355|
|             0.31| 

In [10]:
value_counts_weather_condition = df.groupBy("Weather_Condition").count().orderBy("Weather_Condition")

value_counts_weather_condition.show(200, truncate=False)
value_counts_weather_condition.count()

+-----------------------------------+-------+
|Weather_Condition                  |count  |
+-----------------------------------+-------+
|NULL                               |173459 |
|Blowing Dust                       |189    |
|Blowing Dust / Windy               |266    |
|Blowing Sand                       |2      |
|Blowing Snow                       |740    |
|Blowing Snow / Windy               |925    |
|Blowing Snow Nearby                |1      |
|Clear                              |808743 |
|Cloudy                             |817082 |
|Cloudy / Windy                     |17035  |
|Drifting Snow                      |1      |
|Drifting Snow / Windy              |6      |
|Drizzle                            |4726   |
|Drizzle / Windy                    |23     |
|Drizzle and Fog                    |393    |
|Dust Whirls                        |1      |
|Duststorm                          |3      |
|Fair                               |2560802|
|Fair / Windy                     

145

Wind Chill (F)= measure of how cold the air feels to the human body under windy conditions

In [11]:
Wind_Chill_exploaration=df.select("Wind_Chill(F)","Temperature(F)","Severity")
Wind_Chill_exploaration.show()

+-------------+--------------+--------+
|Wind_Chill(F)|Temperature(F)|Severity|
+-------------+--------------+--------+
|         NULL|          36.9|       3|
|         NULL|          37.9|       2|
|         33.3|          36.0|       2|
|         31.0|          35.1|       3|
|         33.3|          36.0|       2|
|         35.5|          37.9|       3|
|         31.0|          34.0|       2|
|         31.0|          34.0|       3|
|         NULL|          33.3|       2|
|         33.8|          37.4|       3|
|         30.7|          35.6|       3|
|         33.8|          37.4|       3|
|         NULL|          33.8|       2|
|         31.1|          36.0|       2|
|         33.8|          37.4|       2|
|         NULL|          33.8|       2|
|         NULL|          35.6|       2|
|         31.1|          36.0|       2|
|         32.1|          37.4|       2|
|         30.3|          36.0|       2|
+-------------+--------------+--------+
only showing top 20 rows



In [12]:
value_counts_wc = Wind_Chill_exploaration.groupBy("Wind_Chill(F)").count().orderBy("Wind_Chill(F)")

value_counts_wc.show(200)

+-------------+-------+
|Wind_Chill(F)|  count|
+-------------+-------+
|         NULL|1999019|
|         -0.1|    111|
|         -0.2|    399|
|         -0.3|     60|
|         -0.4|    294|
|         -0.5|    127|
|         -0.6|    130|
|         -0.7|     26|
|         -0.8|    246|
|         -0.9|    509|
|         -1.0|   2883|
|         -1.1|    128|
|         -1.3|    274|
|         -1.4|    153|
|         -1.5|     79|
|         -1.6|    135|
|         -1.7|    107|
|         -1.8|    183|
|         -1.9|    152|
|        -10.0|   1660|
|        -10.1|     28|
|        -10.2|     86|
|        -10.3|     77|
|        -10.4|    147|
|        -10.5|     88|
|        -10.6|     66|
|        -10.7|     44|
|        -10.8|     99|
|        -10.9|     47|
|        -11.0|   1450|
|        -11.1|     11|
|        -11.2|     52|
|        -11.3|    128|
|        -11.4|     27|
|        -11.5|     62|
|        -11.6|    158|
|        -11.7|     88|
|        -11.8|     76|
|        -11.9| 

In [13]:
value_counts_wind_direction = df.groupBy("Wind_Direction").count().orderBy("Wind_Direction")
value_counts_wind_direction.show(200)
total_unique_directions = value_counts_wind_direction.count()
print(f"Total unique wind directions: {total_unique_directions}")

+--------------+------+
|Wind_Direction| count|
+--------------+------+
|          NULL|175206|
|          CALM|961624|
|          Calm|368557|
|             E|278914|
|           ENE|258474|
|           ESE|268311|
|          East|103545|
|             N|307151|
|            NE|258639|
|           NNE|255311|
|           NNW|333427|
|            NW|369352|
|         North|153385|
|             S|419989|
|            SE|294901|
|           SSE|349110|
|           SSW|384840|
|            SW|364470|
|         South|177375|
|           VAR|250566|
|      Variable|113996|
|             W|383913|
|           WNW|378781|
|           WSW|353806|
|          West|164751|
+--------------+------+

Total unique wind directions: 25


In [14]:

df_cleaned_wind_condition = df.withColumn("Wind_Direction", upper(col("Wind_Direction")))\
               .withColumn("Wind_Direction", when(col("Wind_Direction") == "VARIABLE", "VAR")\
               .otherwise(col("Wind_Direction")))
# Consolidate directional redundancies
df_cleaned_wind_condition = df_cleaned_wind_condition.withColumn("Wind_Direction", 
                   when(col("Wind_Direction") == "EAST", "E")
                   .when(col("Wind_Direction") == "WEST", "W")
                   .when(col("Wind_Direction") == "NORTH", "N")
                   .when(col("Wind_Direction") == "SOUTH", "S")
                   .when(col("Wind_Direction") == "NORTHEAST", "NE")
                   .when(col("Wind_Direction") == "NORTHWEST", "NW")
                   .when(col("Wind_Direction") == "SOUTHEAST", "SE")
                   .when(col("Wind_Direction") == "SOUTHWEST", "SW")
                   .otherwise(col("Wind_Direction")))

# After consolidation, re-group and count to see the updated distributions
value_counts_weather_direction = df_cleaned_wind_condition.groupBy("Wind_Direction").count().orderBy("Wind_Direction")

# Show the consolidated data
value_counts_weather_direction.show(200)

# Optionally, count the distinct number of wind direction categories after consolidation
print("Distinct Wind Direction Categories:", value_counts_weather_direction.count())

+--------------+-------+
|Wind_Direction|  count|
+--------------+-------+
|          NULL| 175206|
|          CALM|1330181|
|             E| 382459|
|           ENE| 258474|
|           ESE| 268311|
|             N| 460536|
|            NE| 258639|
|           NNE| 255311|
|           NNW| 333427|
|            NW| 369352|
|             S| 597364|
|            SE| 294901|
|           SSE| 349110|
|           SSW| 384840|
|            SW| 364470|
|           VAR| 364562|
|             W| 548664|
|           WNW| 378781|
|           WSW| 353806|
+--------------+-------+

Distinct Wind Direction Categories: 19


In [15]:
zip_exploaration=df.select("Zipcode","State")
zip_exploaration.show()
value_counts_zip = df.groupBy("Zipcode").count().orderBy("Zipcode")
value_counts_zip.show(200)
total_unique_zip = value_counts_zip.count()
print(f"Total unique zipcode: {total_unique_zip}")

+----------+-----+
|   Zipcode|count|
+----------+-----+
|      NULL| 1915|
|     01001|   11|
|01001-1344|    1|
|01001-1832|    1|
|01001-2111|    1|
|01001-2928|    1|
|01001-4605|    1|
|     01002|    8|
|01002-1503|    1|
|01002-1703|    1|
|01002-1811|    1|
|01002-2206|    1|
|01002-2214|    1|
|01002-2240|    3|
|01002-2550|    1|
|01002-2608|    1|
|01002-2631|    1|
|01002-3110|    1|
|01002-3152|    1|
|01002-4008|    1|
|01002-9794|    1|
|     01003|    1|
|     01005|    2|
|01005-8874|    1|
|01005-8898|    2|
|01005-9078|    1|
|01005-9139|    1|
|01005-9150|    1|
|01005-9159|    1|
|01005-9197|    1|
|01005-9248|    1|
|01005-9288|    1|
|01005-9344|    1|
|01005-9392|    1|
|01005-9524|    1|
|01005-9557|    2|
|01005-9558|    1|
|     01007|    4|
|01007-9173|    1|
|01007-9272|    1|
|01007-9301|    1|
|01007-9367|    1|
|01007-9401|    1|
|01007-9430|    1|
|01007-9433|    1|
|01007-9491|    1|
|01007-9524|    1|
|01007-9527|    1|
|01007-9607|    1|
|01007-9626|

In [31]:
zip_exploration = df.select("Zipcode", "City")

# Display the results
zip_exploration.show(500)

+----------+----------------+
|   Zipcode|            City|
+----------+----------------+
|     45424|          Dayton|
|43068-3402|    Reynoldsburg|
|     45176|    Williamsburg|
|     45417|          Dayton|
|     45459|          Dayton|
|     43081|     Westerville|
|45417-2476|          Dayton|
|     45405|          Dayton|
|45404-1923|          Dayton|
|     43081|     Westerville|
|     43228|        Columbus|
|     43068|    Reynoldsburg|
|45420-1863|          Dayton|
|45406-2708|          Dayton|
|43213-1006|        Columbus|
|45410-1721|          Dayton|
|     45402|          Dayton|
|45417-1727|          Dayton|
|45409-2659|          Dayton|
|45406-2640|          Dayton|
|43230-1765|        Columbus|
|45406-4736|          Dayton|
|     45342|      Miamisburg|
|     43211|        Columbus|
|45417-1117|          Dayton|
|43031-9133|       Johnstown|
|45419-4148|          Dayton|
|45405-4326|          Dayton|
|45406-2701|          Dayton|
|     45402|          Dayton|
|43017-255

In [30]:
from pyspark.sql.functions import count

city_grouped = zip_exploaration.groupBy("City").agg(count("Zipcode").alias("Num_Zipcodes"))

# Show the results
city_grouped.show(500)
total_cities_count = city_grouped.count()

print(f"Total number of distinct cities: {total_cities_count}")

+--------------------+------------+
|                City|Num_Zipcodes|
+--------------------+------------+
|             Palermo|         108|
|         Santa Paula|        1196|
|              Osteen|         175|
|            Moreland|         138|
|            Bluffton|        1202|
|              Grimes|         155|
|                Dows|          42|
|           Worcester|         706|
|           Rhinebeck|         693|
|             Hanover|        3048|
|      West Sand Lake|         153|
|              Agawam|          16|
|               Leola|         588|
|        Harleysville|        1549|
|            Rawlings|          15|
|               Tyler|        4080|
|          Westampton|          18|
|       Bowling Green|         610|
|         Springfield|       17347|
|              Corona|       21363|
|         Pine Valley|          66|
|    Oakbrook Terrace|         170|
|          Charleston|        8380|
|          Harrisburg|        5199|
|         Marcus Hook|      

In [33]:
from pyspark.sql.functions import coalesce, first, col
city_mapping = zip_exploaration.filter(zip_exploaration.City.isNotNull()) \
                               .groupBy("Zipcode") \
                               .agg(first("City").alias("MappedCity"))

# Step 2: Join the original DataFrame with the mapping
# and fill in null City values based on the Zipcode
filled_cities_df = zip_exploaration.join(city_mapping, "Zipcode", "left_outer") \
                                   .select(zip_exploaration["*"], 
                                           coalesce(zip_exploaration["City"], city_mapping["MappedCity"]).alias("FilledCity"))

# Replace the original 'City' column with the filled 'FilledCity' values
final_df = filled_cities_df.drop("City").withColumnRenamed("FilledCity", "City")

# Show results
final_df.show(500)
num_null_cities = final_df.filter(col("City").isNull()).count()

print(f"Number of null values in 'City' column after replacement: {num_null_cities}")

+----------+------------------+
|   Zipcode|              City|
+----------+------------------+
|     02125|        Dorchester|
|     02186|            Milton|
|     02747|   North Dartmouth|
|     02886|           Warwick|
|     03060|            Nashua|
|06450-4301|           Meriden|
|     07501|          Paterson|
|     08648|      Lawrence Twp|
|10001-3202|          New York|
|10016-3539|          New York|
|     10452|             Bronx|
|10510-2305|  Briarcliff Manor|
|     10523|          Elmsford|
|     10523|          Elmsford|
|     10523|          Elmsford|
|     10603|      White Plains|
|     10603|      White Plains|
|     10710|           Yonkers|
|     10801|      New Rochelle|
|     10975|       Southfields|
|     11749|          Islandia|
|     12144|        Rensselaer|
|     12144|        Rensselaer|
|     12204|            Albany|
|     12204|            Albany|
|     12211|            Albany|
|     12508|            Beacon|
|     12529|         Hillsdale|
|     14